# TP4 : Développement du plateau du Heysel entre 1847 et 1978

## Préparation du corpus

## Imports

In [23]:
import json
import requests
import os
import re
import time
from collections import defaultdict
import sys
import spacy
from collections import Counter
from wordcloud import WordCloud
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image
import mmap
from difflib import SequenceMatcher
import openpyxl
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
nlp = spacy.load('fr_core_news_md')

In [3]:
#obtenir le nom des rue sur nominatim pour le chaque latitude et longitude du plateau du Heysel et les écrire dans mon fichier de recherche

fichier = open("../data/rues.txt", "a")

latmmax = 50.90350397899814
longmin = 4.328493534801743
latmin= 50.88509135160375
longmax = 4.354090559446083

while longmin <= longmax:
    longmin=longmin+0.0010000000000000
    x=latmin
    while x <= latmmax:
        url = "https://nominatim.openstreetmap.org/reverse"
        data = {'lat': x, 'lon': longmin,'format': 'json'}
        resp = requests.get(url, data)
        json_list = json.loads(resp.text)
        try:
            fichier.write(json_list["address"]["road"] + "\n")
        except KeyError:
            fichier.write("erreur rue" + "\n")
        x=x+0.0050000000000
 
    
fichier.close()

In [4]:
#Retirer les doublons, Esplanade, A212 et erreur rue du fichier de recherche
ls = []

with open("../data/rues.txt", 'r') as file:
    for line in file:
        if line.lower() not in ls and line!="erreur rue"+"\n" and line!="Esplanade"+"\n" and line!="A12"+"\n":
            ls.append(line.lower())

with open("../data/rues.txt", 'w') as file:
    for line in ls:
        file.write(line)
        
file.close()

In [6]:
#mettre un nom de rue par ligne dans une langue (FR ou NL)
ls = []
stopwords1=("jean joseph", "jean", "joseph", "arthur", "ernest", "gustave", "adrien", "guillaume", "edouard", "léopold", "leopold", "jean", "baptiste", "reper", "moens", "sentier", 
               "smet", "félix", "felix", "stevens", "pierre", "emile", "avenue", "place", "rue ","de l'", "de la ", "van ", "boulevard", "chaussée", "des ", " la ", "gros", "laeken",
           "anne", "belgique", "baptiste ", "de ", "hêtres ", "du ", "sports", "square", "sainte", "saint", "compte", "graaf", "drève", "parc", "royal", "koninklijk", "palais")

with open("../data/rues.txt", 'r') as file:
    for line in file:
        try:
            elems = line.split("-")
            FR = elems[0]
            NL = elems[1]
            for mot in stopwords1:
                if mot in FR:
                    FRshort= FR.replace(mot,'')
                    FR=FRshort
                if mot in NL:
                    NLshort= NL.replace(mot,'')
                    NL=NLshort
        except KeyError:
            ls.append(line)
        ls.append(FR.strip() + "\n")
        ls.append(NL.strip() + "\n")
file.close()
            
ls.append("esplanade\n")            
ls.append("atomium\n")
ls.append("mini-europe\n")   
ls.append("heysel\n") 
ls.append("expositions\n")
ls.append("exposition\n")
ls.append("expo\n") 
ls.append("trademart\n") 
ls.append("brupark\n") 
ls.append("planétarium\n") 
ls.append("stade\n")
ls.append("meli\n")
ls.append("universelle\n") 
ls.append("stevens-delannoy\n")
ls.append("reper-vreven\n")
ls.append("saint-lambert\n")
ls.append("sainte-anne\n")
ls.append("stuyvenberg\n")
ls.append("château\n")
ls.append("belvédère\n")
ls.append("kasteel\n")
ls.append("plateau\n")
ls.append("japonaise\n")
ls.append("chinois\n")
ls.append("moyen-orient\n")

           
with open("../data/rues.txt", 'w') as file:
    for line in ls:
        if line!="\n":
            file.write(line)     
file.close()

ls=[]
with open("../data/rues.txt", 'r') as file:
    for line in file:
        if line not in ls and line!="\n":
            ls.append(line)

with open("../data/rues.txt", 'w') as file:
    for line in ls:
        file.write(line)

In [7]:
#vérifier le contenu du fichier de recherche
fichier = open("../data/rues.txt", "r")
print (fichier.read())
fichier.close()

crocq
crocqlaan
gehuchten
gehuchtenplein
disque
schijfstraat
romaine
romeinsesteenweg
masoin
masoinlaan
rommelaere
rommelaerelaan
houba
strooper
cresson
waterkersstraat
gilson
gilsonstraat
sportlaan
ciboulette
bieslookstraat
bayet
bayetlaan
genévriers
jeneverbomenstraat
greef
greeflaan
kufferath
kufferathlaan
marathon
marathonlaan
madrid
madridlaan
peret
peretstraat
depaire
centenaire
eeuwfeestlaan
vreven
salu
salustraat
naeyer
naeyerlaan
science
wetenschapslaan
heymans
heymansstraat
sterckx
sterckxstraat
delannoy
atomium
atomiumplein
strauwen
strauwenstraat
wauters
wautersstraat
hallier
kreupelboslaan
clémentine
clementina
lambert
ebéniers
ebbebomenlaan
passerelle
loopbruglaan
trembles
abelenlaan
robiniers
witte
pourpres
puperbeukenlaan
setter
setterweg
médori
médoristraat
seringas
wildejasmijnenlaan
fernig
ferniglaan
tilleul
dikkelindelaan
amphore
amforalaan
dynastie
vorstenhuisplein
vorstenhuislaan
forum
forumlaan
parklaan
esplanade
mini-europe
heysel
expositions
exposition
expo
tra

In [8]:
#on met les fichiers txt en minuscule pour faciliter la recherche et on les sauvegarde sous data/tmp
ls = []

temp_path = '../data/tmp'
txt_path = '../data/txt'

for f in os.listdir(txt_path): 
    if os.path.isfile(os.path.join(txt_path, f)):
        bulletin=txt_path + "/" + f
        with open(bulletin, 'r') as filein:
            for line in filein:
                ls.append(line.lower())
    filein.close()
    bulletin_clean=temp_path + "/" + f   
    with open(bulletin_clean, 'w') as fileout:
        for line in ls:
            fileout.write(line)
    fileout.close()
    ls = []



In [9]:
#on fait une liste de mots à corriger dans le corpus et on les mets dans fichier correction.txt - ici on va corriger les espaces dans les mots à rechercher
ls = []
mot=[]
with open("../data/rues.txt", 'r') as file:
    for line in file:
        string1=line.strip()
        for lettre in string1:
            mot.append(lettre)
        ls.append(" ".join(mot)+" = "+ string1 + "\n")
        mot=[]


with open("../data/correction.txt", 'w') as fileout:
    for line in ls:
        fileout.write(line)



In [10]:
#on parcourt les bulletins et on fait les corrections nécéssaires.
txt_path = '../data/tmp'
ls=[]
compteur=0

with open("../data/correction.txt", 'r') as filecorr:
    for corr in filecorr:
        try:
            elems = corr.split("=")
            erreur = elems[0]
            correction = elems[1]    
            for f in os.listdir(txt_path): 
                if os.path.isfile(os.path.join(txt_path, f)):
                    bulletin=txt_path + "/" + f
                    with open(bulletin, 'r') as filein:
                        for line in filein:
                            if erreur in line:
                                line=line.replace(erreur,correction)
                                compteur+=1
                            ls.append(line)
                    with open(bulletin, 'w') as fileout:
                        for line in ls:
                            fileout.write(line)
                    ls=[]
            print(str(compteur) + " " + erreur + " corrigé par " + correction)
            compteur=0
        except KeyError:
            print("erreur")
                                

23c r o c q  corrigé par  crocq

0c r o c q l a a n  corrigé par  crocqlaan

5g e h u c h t e n  corrigé par  gehuchten

0g e h u c h t e n p l e i n  corrigé par  gehuchtenplein

94d i s q u e  corrigé par  disque

0s c h i j f s t r a a t  corrigé par  schijfstraat

103r o m a i n e  corrigé par  romaine

0r o m e i n s e s t e e n w e g  corrigé par  romeinsesteenweg

4m a s o i n  corrigé par  masoin

0m a s o i n l a a n  corrigé par  masoinlaan

32r o m m e l a e r e  corrigé par  rommelaere

0r o m m e l a e r e l a a n  corrigé par  rommelaerelaan

250h o u b a  corrigé par  houba

36s t r o o p e r  corrigé par  strooper

4c r e s s o n  corrigé par  cresson

0w a t e r k e r s s t r a a t  corrigé par  waterkersstraat

69g i l s o n  corrigé par  gilson

0g i l s o n s t r a a t  corrigé par  gilsonstraat

0s p o r t l a a n  corrigé par  sportlaan

0c i b o u l e t t e  corrigé par  ciboulette

0b i e s l o o k s t r a a t  corrigé par  bieslookstraat

8b a y e t  corrigé pa

In [14]:
wb = Workbook()

dest_filename = '../data/frequences.xlsx'

ws1 = wb.active
ws1.title = "frequence"
ws1['A1'] = "bulletin"
ws1['B1'] = "commune"
ws1['C1'] = "année"
col=4
with open("../data/rues.txt", 'r') as file:
    for line in file:
        lettre=get_column_letter(col)
        ws1[lettre + str(1)]= line
        col+=1
        
        
txt_path = '../data/tmp'
row=2
for f in os.listdir(txt_path): 
    if os.path.isfile(os.path.join(txt_path, f)):
        ws1['A' + str(row)]= f
        fname=txt_path + "/" + f
        if "_" in f :
            elems = f.split("_")
            city = elems[0]
            year = elems[1]
            ws1['B' + str(row)]= city
            ws1['C' + str(row)]= year
        frequencies = Counter(open(fname,'r').read().split())
        col=4
        with open("../data/rues.txt", 'r') as recherche:
            for line in recherche:
                lettre=get_column_letter(col)
                ws1[lettre + str(row)]= frequencies[line.strip()]
                col+=1
        row+=1
                
maxrow=len(ws1['A'])
maxcol=ws1.max_column
ws1['A'+ str(maxrow+1)]="Total frequence"
total=0
col=4
while col <= maxcol:
    lettre=get_column_letter(col)
    for row in range(2,maxrow):
        total+=int(ws1[lettre + str(row)].value)
    ws1[lettre + str(maxrow+1)]=total
    if total==0:
#        print((ws1[lettre + str(1)].value).strip() + " éliminé")
        ws1.delete_cols(col)
        col=col-1
        maxcol=maxcol-1
    total=0
    col+=1



lettre1=get_column_letter(maxcol+1)
ws1[lettre1+ str(1)]="Total frequence"
total=0
row=2
while row <=maxrow:
    for col in range(4,maxcol):
        lettre=get_column_letter(col)
        total+=ws1[lettre + str(row)].value
    ws1[lettre1 + str(row)]=total
    if total<=10:
#        print((ws1['A' + str(row)].value).strip() + " éliminé")
        ws1.delete_rows(row)
        row=row-1
        maxrow=maxrow-1
    total=0
    row+=1

wb.save(dest_filename) 
  



In [15]:
## Filtrer le corpus à l'aide des mots mis dans la liste de recherche

In [21]:
# Rassembler tous ces fichiers en un seul document tp4_all.txt en ajoutant un marquage pour indiquer sur quelle année porte le bulletin. 
#Nous éliminons au passage tous les bulletins de Bruxelles antérieurs à 1920 

filename = '../data/frequences.xlsx'
txt_path = '../data/tmp'
content_list = []
workbook = openpyxl.load_workbook(filename)
ws1 = wb.active
compteur=0

maxrow=(len(ws1['A']))-1
maxcol=(ws1.max_column)-1
row=2
while row <=maxrow-1:
    string_deb="$$$$$$$$$$" + "\n"
    file=ws1['A' + str(row)].value
    city=ws1['B' + str(row)].value
    year=ws1['C' + str(row)].value
    if (city=="Bxl" and int(year)>=1920) or city=="Lkn":
        content_list.append("$$$$$$$$$$"+ year + "\n")
        compteur+=1
    with open(os.path.join(txt_path, file), 'r') as bulletin:
        content_list.append(bulletin.read())
    row+=1    

print(str(compteur))
temp_path = '../data'
with open('../data/tp4_all.txt', 'w') as f:
    f.write(' '.join(content_list))

703


## Nettoyer le fichier à l'aide d'une fonction de nettoyage

In [25]:
# Stopwords (Idem que dans s1) _ repris du TP3
sw = stopwords.words("french")
sw += ["abord", "acceptée", "accord", "accorder", "acte", "action", "actuel", "adjudication", "administration", "admis", "adopté", "adoptées", "adoption", 
       "adresse", "adressées", "affaire", "afin", "agit", "ailleurs", "ainsi", "ajouter", "alors", "amendement", "anderlecht", "année", "années", "annuel", "ans", "août", 
       "appartenant", "appel", "application", "approbation", "approuvé", "après", "ares", "argent", "arrêté", "art", "article", "articles", "assez", 
       "assurer", "aucun", "aucune", "augmenter", "aujourd", "aussi", "autant", "auteur", "autorisation", "autorisé", "autoriser", "autorité", "autre", 
       "autres", "avant", "avenir", "avenue", "avis", "avoir", "avril", "bas", "base", "bases", "bâtiment", "bâtiments", "bâtisses", "beaucoup", "beaux", 
       "besoin", "besoins", "bien", "bienfaisance", "biens", "bon", "boulangerie", "boulevard", "bourgmestre", "bruxelles", "bulletin", "bureau", "bureaux", 
       "but", "caisse", "car", "carrés", "cas", "cause", "cela", "celle", "celles", "celte", "celui", "cent", "centiares", "centimes", "centimètres", 
       "cependant", "certain", "certaines", "certains", "cet", "cette", "ceux", "chacun", "chapitre", "chaque", "charge", "charger", "charges", "charité", 
       "chaussée", "chef", "chez", "chiffre", "chiffres", "chose", "choses", "cinq", "circonstances", "classe", "collège", "collègue", "comité", "comme", 
       "comment", "commission", "communal", "communale", "communales", "communaux", "commune", "communes", "compagnie", "compagnies", "compris", "compte", 
       "concerne", "concession", "conclusion", "conclusions", "condition", "conditions", "conformément", "connaissance", "connaître", "conseil", 
       "conseillers", "conséquence", "considérable", "considérables", "considère", "contenance", "contentieux", "contraire", "contrat", "contre", "convention", "côté", 
       "courant", "cours", "créances", "créer", "crois", "dame", "date", "décembre", "décembre.", "décision", "déclaration", "dehors", "déjà", "delà", 
       "délai", "délibération", "demande", "demandé", "demander", "demandes", "demeurant", "dépôt", "depuis", "députation", "dêputation", "dernier", 
       "dernière", "dès", "dessus", "deux", "deuxième", "devant", "devis", "devoir", "devons", "devront", "différence", "difficile", "difficultés", 
       "dire", "direction", "discussion", "disposition", "dispositions", "dit", "divers", "diverses", "division", "dix", "doit", "doivent", "donation", 
       "donc", "donne", "donné", "donnée", "donner", "dont", "double", "doute", "droits", "echevin", "échevin", "echevins", "échevins", "effet", "également", 
       "égard", "église", "élevé", "elles", "émettre", "emplacement", "emploi", "encore", "enfin", "engage", "enquête", "ensemble", "ensuite", "entendu", 
       "entre", "entrée", "entreprise", "environ", "époque", "ester", "établi", "établir", "établissement", "établissements", "etat", "état", "etc", "être", 
       "etterbeek", "étude", "évaluation", "évaluations", "examen", "exécuter", "exécution", "exemple", "exercice", "existe", "extraordinaires", "fabrique", "face", 
       "facile", "façon", "faire", "fait", "faite", "faites", "faits", "famille", "faudrait", "faut", "favorable", "fera", "février", "filles", "fixé", 
       "fois", "fonds", "font", "forme", "fort", "frais", "francs", "garde", "général", "générale", "grand", "grande", "grandes", "grands", "grandvelle", 
       "habitants", "habitations", "haut", "haute", "herpels", "het", "heure", "heures", "hommes", "honneur", "honorable", "hospice", "hospices", "hôtel", "hui", 
       "huit", "ici", "idée", "idem", "ils", "immédiatement", "immeuble", "immeubles", "impasse", "impasses", "importance", "impossible", "inconvénient", "inconvénients", 
       "indispensable", "industrie", "informe", "inscription", "instruction", "intenter", "intérêt", "intérêts", "jamais", "janvier", "jean", "jour", "jours", "juillet", 
       "juin", "jusqu", "juste", "justice", "kil", "laeken", "laisser", "laquelle", "lecture", "legs", "lequel", "les", "lettre", "lettres", "leurs", "libre", "lieu", 
       "liste", "lit", "local", "location", "locaux", "loi", "loin", "longtemps", "lors", "lorsqu", "lorsque", "lot", "mai", "maintenant", "maison", 
       "maisons", "malgré", "manière", "marchai", "mars", "matériaux", "matières", "matin", "membre", "membres", "mêmes", "ménage", "messieurs", "mesures", 
       "mètre", "mètres", "mettre", "mieux", "mille", "millions", "minimes", "mis", "moins", "mois", "moment", "montant", "mots", "mouvement", "moyen", 
       "moyenne", "moyens", "nature", "nécessaire", "nécessaires", "nécessité", "neuf", "noires", "nom", "nombre", "nombreux", "non", "notaire", "nouveau", 
       "nouveaux", "nouvelle", "nouvelles", "novembre", "objet", "objets", "observation", "observations", "obtenir", "obtenu", "occasion", "octobre", 
       "officiers", "onze", "opinion", "ordinaires", "ordonnance", "ordre", "outre", "ouverte", "ouverture", "paiement", "pain", "paraît", "parce", 
       "parfaitement", "parmi", "part", "partie", "parties", "partir", "passé", "payer", "pays", "pendant", "pense", "permanente", "perpétuité", "personne", "personne", 
       "personnel", "personnelle", "personnes", "petit", "petite", "peu", "peut", "peuvent", "pièces", "pierre", "pierres", "place", "plus", "plusieurs", 
       "point", "populaire", "population", "portant", "porté", "portée", "position", "possible", "pourquoi", "pourra", "pourrait", "pouvoir", 
       "pouvons", "premier", "première", "prendre", "près", "présence", "présent", "présente", "présenté", "présenter", "présents", "principal", 
       "principale", "principe", "pris", "procédé", "procès", "prochain", "prochaine", "produit", "profit", "proportion", "propose", "proposé", "proposer", 
       "proposition", "propositions", "propriétaire", "propriétaires", "propriété", "propriétés", "provincial", "provisoire", "public", "publics", "publique", 
       "publiques", "puis", "puisque", "puisse", "pyck", "quand", "quant", "quartier", "quatre", "quelque", "quelques", "question", "questions", "quoi", "raison", 
       "rapport", "rapports", "receveur", "recevoir", "réclamation", "réclamations", "reçu", "règlement", "relatif", "relatives", "remarquer", 
       "remboursement", "rendre", "renseignements", "rentes", "renvoi", "résolution", "ressources", "reste", "résultat", "résulte", "rien", 
       "rue", "rues", "saint", "sainte", "salle", "salles", "sans", "sauf", "savoir", "seance", "séance", "second", "seconde", "secours", 
       "secrétaire", "secrétariat", "section", "sections", "semble", "sens", "sept", "septembre", "sépulture", "service", "seul", "seule", 
       "seulement", "sieur", "simple", "sise", "situation", "situé", "situés", "six", "société", "sociétés", "sœurs", "sollicite", "solution", 
       "sorte", "soumettre", "soumis", "soumission", "soumissions", "sous", "souvent", "suit", "suite", "suivant", "suivante", "suivantes", "suivants", 
       "sujet", "supérieure", "supplémentaire", "surtout", "système", "tableau", "tandis", "tant", "tard", "tel", "telle", "temps", "tendant", 
       "termes", "terrain", "terrains", "territoire", "titre", "total", "toujours", "tous", "tout", "toute", "toutes", "transmettre", "travai", 
       "travail", "très", "trois", "troisième", "trop", "trouve", "trouvent", "trouver", "trouvés", "unanimité", "usage", "utilité", "valeur", 
       "van", "vanhelst", "vend", "vendre", "verbal", "verbal", "vers", "veuve", "vient", "vieux", "ville", "villes", "vingt", "vis", "voici", 
       "voir", "voix", "volume", "vote", "voté", "voter", "voyez", "vrai", "vue"]
sw = set(sw)


In [28]:
def clean_text(filein,fileout):
    input_path = filein
    output_path = fileout
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [29]:
clean_text('../data/tp4_all.txt', '../data/tp4_all_clean.txt')

'Output has been written in ../data/tp4_all_clean.txt!'

## Nuage de mots

### Afficher les termes les plus fréquents


In [31]:
with open('../data/tp4_all_clean.txt', 'r') as f:
    after = f.read()

frequencies = Counter(after.split())

print(frequencies.most_common(200))

[('travaux', 30364), ('budget', 26262), ('intervention', 17250), ('ecole', 16207), ('dépenses', 15624), ('dépense', 14446), ('recettes', 13936), ('police', 12709), ('assistance', 12636), ('prix', 12531), ('projet', 12486), ('enfants', 12351), ('école', 11898), ('enseignement', 11388), ('nomination', 11212), ('monsieur', 11067), ('construction', 10929), ('crédit', 10500), ('professeur', 10414), ('een', 10412), ('mesdames', 10369), ('écoles', 10336), ('services', 9528), ('voor', 9314), ('élèves', 9290), ('entretien', 9132), ('taxe', 8981), ('somme', 8921), ('gaz', 8796), ('scolaire', 8641), ('plan', 7875), ('vente', 7856), ('dat', 7759), ('droit', 7741), ('mise', 7583), ('considérant', 7540), ('hôpital', 7176), ('aan', 7078), ('subside', 6908), ('exposition', 6855), ('royal', 6506), ('voie', 6258), ('institut', 6241), ('primaire', 6111), ('finances', 6088), ('eau', 6083), ('aménagement', 6075), ('fin', 6048), ('augmentation', 5972), ('fonctions', 5915), ('création', 5879), ('die', 5837),

### Créer, stocker et afficher le nuage de mots

In [ ]:
cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"{year}.png"))
Image(filename=os.path.join(temp_path, f"{year}.png"))

In [32]:
#reconnaissance d'entités nommée par très utile pour cet exercice

n=900000
text = open("../data/tp4_all_clean.txt", encoding='utf-8').read()[:n]
doc = nlp(text)
localite = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "LOC" and len(ent.text) > 3:
        localite[ent.text] += 1
        
sorted_localite = sorted(localite.items(), key=lambda kv: kv[1], reverse=True)

for loca, freq in sorted_localite[:50]:
    print(f"{loca} apparait {freq} fois dans le corpus")

belgique apparait 21 fois dans le corpus
paris apparait 16 fois dans le corpus
namur apparait 15 fois dans le corpus
luxembourg apparait 12 fois dans le corpus
gand apparait 9 fois dans le corpus
derammeleer apparait 9 fois dans le corpus
josse apparait 8 fois dans le corpus
ixelles apparait 8 fois dans le corpus
pachéco apparait 8 fois dans le corpus
athénée apparait 8 fois dans le corpus
france apparait 7 fois dans le corpus
londres apparait 7 fois dans le corpus
maelbeék apparait 6 fois dans le corpus
étang josse ten noode apparait 6 fois dans le corpus
étang josse ten apparait 5 fois dans le corpus
bogards apparait 5 fois dans le corpus
steenpoort apparait 5 fois dans le corpus
somme apparait 4 fois dans le corpus
cologne apparait 4 fois dans le corpus
decoen apparait 4 fois dans le corpus
waterloo apparait 4 fois dans le corpus
anvers apparait 3 fois dans le corpus
jaugeages apparait 3 fois dans le corpus
new york apparait 3 fois dans le corpus
vanderelst apparait 3 fois dans le c